## Haiti Health Data Analysis

You are an analyst working at a health research company in Haiti.  The government has asked your company to analyze data related to covid-19.  To that end, they have provided you with access to health data.

Your goal as a Data Manager is to provide
*  Relational Database to manage all the data collection
* Use SQL Language to answers some query provided from the Board
* Build a dashboard with Power BI to illustrate all the activites of covid


### Importing Libraries

In [113]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

### Loading Dataset Files

In [115]:
covid_cases = pd.read_csv("./datasets/mspp_covid19_cases.csv", index_col=0)
spa         = pd.read_csv("./datasets/spa.csv",index_col=0)
departement = pd.read_excel("./datasets/hti_adminboundaries_tabulardata.xlsx",sheet_name=1,index_col=0)
commune     = pd.read_excel("./datasets/hti_adminboundaries_tabulardata.xlsx",sheet_name=2,index_col=0)

### Data Cleaning

In [ ]:
# Display Covid_cases Informations
covid_cases.info()

In [117]:
#Drop total columns from covid_cases data files
covid_cases=covid_cases.drop([10,21,32,43,54,65,76,87,98])
covid_cases=covid_cases.sort_values(by=['document_date','departement'],ascending=[True,True])

In [ ]:
# Replace wrong data from covid_cases departement column
covid_cases['departement'] = covid_cases['departement'].replace({'Guest' :'Ouest'}, inplace =False)
covid_cases.info()

In [119]:
#Merge Departement with covid_cases
covid_cases1 = pd.merge(covid_cases,departement,how= 'left', left_on='departement', right_on='adm1_fr')
covid_cases1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90 entries, 0 to 89
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   departement             90 non-null     object 
 1   cas_suspects            90 non-null     int64  
 2   cas_confirmes           90 non-null     int64  
 3   deces                   90 non-null     int64  
 4   taux_de_letalite        90 non-null     float64
 5   document_date           90 non-null     object 
 6   adm0_en                 90 non-null     object 
 7   adm0_fr                 90 non-null     object 
 8   adm0_ht                 90 non-null     object 
 9   adm1code                90 non-null     object 
 10  adm1_en                 90 non-null     object 
 11  adm1_fr                 90 non-null     object 
 12  adm1_ht                 90 non-null     object 
 13  IHSI_UNFPA_2019         90 non-null     float64
 14  IHSI_UNFPA_2019_female  90 non-null     int6

In [ ]:
#selecting the necessary columns in covid_cases data
covid_cases1= covid_cases1.loc[:,['cas_suspects','cas_confirmes','deces','taux_de_letalite','document_date','adm1code']]
covid_cases1.info()

In [122]:
departement.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, HT to HT
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   adm0_en                 10 non-null     object 
 1   adm0_fr                 10 non-null     object 
 2   adm0_ht                 10 non-null     object 
 3   adm1code                10 non-null     object 
 4   adm1_en                 10 non-null     object 
 5   adm1_fr                 10 non-null     object 
 6   adm1_ht                 10 non-null     object 
 7   IHSI_UNFPA_2019         10 non-null     float64
 8   IHSI_UNFPA_2019_female  10 non-null     int64  
 9   IHSI_UNFPA_2019_male    10 non-null     int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 880.0+ bytes


In [123]:
##selecting the necessary columns in departement data
departement=departement.loc[:,['adm1code','adm1_en','IHSI_UNFPA_2019','IHSI_UNFPA_2019_female','IHSI_UNFPA_2019_male']]
departement.head()

,adm1code,adm1_en,IHSI_UNFPA_2019,IHSI_UNFPA_2019_female,IHSI_UNFPA_2019_male
adm0code,,,,,
HT,HT01,West,5.010206e+06,2587360,2422846
HT,HT02,South-East,8.939007e+05,448111,445790
HT,HT03,North,1.159762e+06,589825,569932
HT,HT04,North-East,4.444631e+05,221501,222966
HT,HT05,Artibonite,2.047153e+06,1034065,1013085


In [124]:
commune=commune.reset_index()
display(commune.head())

,adm0code,adm0_en,adm0_fr,adm0_ht,adm1code,adm1_en,adm1_fr,adm1_ht,adm2code,adm2_en,adm2_fr,adm2_ht,IHSI_UNFPA_2019,IHSI_UNFPA_2019_female,IHSI_UNFPA_2019_male
0,HT,Haiti,Haïti,Ayiti,HT01,West,Ouest,Lwès,HT0111,Port-au-Prince,Port-au-Prince,,1.227540e+06,648353,579188
1,HT,Haiti,Haïti,Ayiti,HT01,West,Ouest,Lwès,HT0112,Delmas,Delmas,,4.914342e+05,258526,232908
2,HT,Haiti,Haïti,Ayiti,HT01,West,Ouest,Lwès,HT0113,Carrefour,Carrefour,,6.357646e+05,333500,302263
3,HT,Haiti,Haïti,Ayiti,HT01,West,Ouest,Lwès,HT0114,Petion-Ville,Pétion-Ville,,4.685251e+05,241407,227118
4,HT,Haiti,Haïti,Ayiti,HT01,West,Ouest,Lwès,HT0115,Kenscoff,Kenscoff,,7.140848e+04,35608,35800


In [125]:
#selecting the necessary columns in commune data
commune=commune.loc[:,['adm1code','adm2code','adm2_en','IHSI_UNFPA_2019','IHSI_UNFPA_2019_female','IHSI_UNFPA_2019_male']]
commune.head()

,adm1code,adm2code,adm2_en,IHSI_UNFPA_2019,IHSI_UNFPA_2019_female,IHSI_UNFPA_2019_male
0,HT01,HT0111,Port-au-Prince,1.227540e+06,648353,579188
1,HT01,HT0112,Delmas,4.914342e+05,258526,232908
2,HT01,HT0113,Carrefour,6.357646e+05,333500,302263
3,HT01,HT0114,Petion-Ville,4.685251e+05,241407,227118
4,HT01,HT0115,Kenscoff,7.140848e+04,35608,35800


### Spa cleaning

In [126]:
spa=spa.reset_index()
spa.head()

,level_0,index,facil,depart,departn,vilcom,vilcomn,factype,facdesc_1,facdesc,mga,service_laboratory,num_beds,ambulance,full_time
0,0,0,1,1,Ouest,11,Port-Au-Prince,7.0,DISPENSAIRE,DISPENSAIRE,3,1.0,NaN,3.0,5.0
1,1,1,2,1,Ouest,11,Port-Au-Prince,3.0,HOPITAL,HOPITAL COMMUNAUTAIRE DE REFERENCE,1,1.0,25.0,3.0,18.0
2,2,2,3,1,Ouest,11,Port-Au-Prince,6.0,CENTRE DE SANTE SANS LIT,CENTRE DE SANTE SANS LIT,3,1.0,201.0,1.0,141.0
3,3,3,4,1,Ouest,11,Port-Au-Prince,5.0,CENTRE DE SANTE AVEC LIT,CENTRE DE SANTE AVEC LIT,3,1.0,7.0,3.0,13.0
4,4,4,5,1,Ouest,11,Port-Au-Prince,6.0,CENTRE DE SANTE SANS LIT,CENTRE DE SANTE SANS LIT,2,1.0,NaN,3.0,10.0


In [ ]:
pd.set_option('display.max_rows',None)
x = spa['vilcomn'].drop_duplicates()

In [ ]:
b = commune.loc[:,'adm2_en']
b = b.reset_index() 
b

In [129]:
spa['vilcomn']= spa['vilcomn'].replace({"L!og!ne":"Leogane","Limb!":"Limbe","St-Ls-du-Nord'Ouest"	:"Saint-Louis du Nord" ,"Cayes" :"Les Cayes" }, inplace =False)

In [ ]:
from rapidfuzz import process
t = spa['vilcomn'].map(lambda x: process.extractOne(x, b.iloc[:,1]))
t

In [131]:
def listed( t) : 
    list1 = []
    a=0
    for i in range(0,t.shape[0]) :
        a = list(t[i])
        list1.append(a[0]) 
    spa['new'] = list1    
    return spa['new']             

In [ ]:
listed(t)

In [133]:
#merge spawith commune
f_merge=pd.merge(left=spa, right=commune, how='left', left_on='new', right_on='adm2_en')

In [134]:
#creating new table as newtab 
newtab= spa.loc[:,['factype','facdesc_1','facdesc']]
newtab=newtab.drop_duplicates().sort_values(by='factype')
newtab.head()

,factype,facdesc_1,facdesc
16,1.0,HOPITAL,HOPITAL UNIVERSITAIRE
289,2.0,HOPITAL,HOPITAL DEPARTEMENTAL
1,3.0,HOPITAL,HOPITAL COMMUNAUTAIRE DE REFERENCE
12,4.0,HOPITAL,AUTRES HOPITAUX
3,5.0,CENTRE DE SANTE AVEC LIT,CENTRE DE SANTE AVEC LIT


In [135]:
f_merge.columns

Index(['level_0', 'index', 'facil', 'depart', 'departn', 'vilcom', 'vilcomn',
       'factype', 'facdesc_1', 'facdesc', 'mga', 'service_laboratory',
       'num_beds', 'ambulance', 'full_time', 'new', 'adm1code', 'adm2code',
       'adm2_en', 'IHSI_UNFPA_2019', 'IHSI_UNFPA_2019_female',
       'IHSI_UNFPA_2019_male'],
      dtype='object')

In [136]:
# Selecting the needed column from the first merge data
f_mergeclean = f_merge.loc[:,['factype','mga','ambulance','service_laboratory', 'num_beds',
       'ambulance', 'full_time', 'adm1code', 'adm2code']]

In [137]:
f_mergeclean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1033 entries, 0 to 1032
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   factype             1033 non-null   float64
 1   mga                 1033 non-null   int64  
 2   ambulance           1007 non-null   float64
 3   service_laboratory  1007 non-null   float64
 4   num_beds            474 non-null    float64
 5   ambulance           1007 non-null   float64
 6   full_time           1007 non-null   float64
 7   adm1code            1033 non-null   object 
 8   adm2code            1033 non-null   object 
dtypes: float64(6), int64(1), object(2)
memory usage: 80.7+ KB


In [139]:
#Loading data to mysql database
engine = create_engine('mysql://root:@localhost/covid')
with engine.connect() as conn, conn.begin():
    f_mergeclean.to_sql("spa",conn, if_exists="replace",index=True)
    newtab.to_sql("factype",conn, if_exists="replace",index=True)
    covid_cases1.to_sql("covid_case",conn, if_exists="replace",index=True)
    commune.to_sql("commune",conn, if_exists="replace",index=True)
    departement.to_sql("departement",conn, if_exists="replace",index=True)